In [75]:
import numpy as np
import time

In [76]:
def HO_SVD(A, r1, r2, r3):
    n1,n2,n3 = A.shape
    #print(A.shape)
    A1 = A.reshape(n1, n2*n3)
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V1 = U[:,:r1] 
    V1 = V1.T
    #print("V1 shape", V1.shape, np.linalg.norm(V1, ord=2))
    A1 = np.diag(s[:r1]) @ V[:r1,:] # r1 x n2*n3
    A1 = A1.reshape(r1, n2, n3)
    A1 = A1.transpose([0,2,1])
    A1 = A1.reshape(r1 * n3, n2)# r1 * n3 x n2
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V2 =  V[:r2,:] # r2 x n2
    #print("V2 shape", V2.shape, np.linalg.norm(V2, ord=2))
    U2 = U[:,:r2] @ np.diag(s[:r2]) # r1*n3 x r2
    U2 = U2.reshape(r1, n3, r2) # r1 x n3 x r2
    U2 = U2.transpose([0,2,1]) # r1 x r2 x n3
    U2 = U2.reshape(r1 * r2, n3) # r1*r2 x n3
    U, s, V = np.linalg.svd(U2, full_matrices=False)
    V3 = V[:r3,:] # U3(r3,j)
    #print("V3 shape", V3.shape, np.linalg.norm(V3, ord=2))
    G = U[:,:r3] @ np.diag(s[:r3])# r1 * r2 x r3
    #print("main core shape", G.shape)
    G = G.reshape(r1, r2, r3) # main core
    return G, V1, V2, V3

def Check(A, G, V1, V2, V3):
    # || A - A_tucker||_F 
    err = 0
    r1, r2, r3 = G.shape
    A1 = G.reshape(r1 * r2, r3).dot(V3) # r1*r2 x n3
    A1 = A1.reshape(r1, r2, V3.shape[1]).transpose([0,2,1]) # r1 x r2 x n3
    A1 = A1.reshape(r1*V3.shape[1], r2).dot(V2) # r1*n3 x n2
    A1 = A1.reshape(r1, V3.shape[1], V2.shape[1]).transpose([1,2,0]) # n3 x n2 x r1
    A1 = A1.reshape(V3.shape[1], V2.shape[1], r1).dot(V1) # n3 x n2 x n1
    A1 = A1.transpose([2,1,0]) # n1 x n2 x n3
    return np.linalg.norm(A - A1) #error norm

In [77]:
%%time

N = 64
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
G, V1, V2, V3 = HO_SVD(A,4,4,4)
print("Error =", Check(A, G, V1,V2,V3))

Error = 1.5886924857596471e-12
CPU times: user 300 ms, sys: 23.3 ms, total: 323 ms
Wall time: 335 ms


In [70]:
%%time

N = 32
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.cos(i+2.*j+3.*k)
G, V1, V2, V3 = HOSVD_3d(A,2,2,2)
print("Error =", compute_err_np_tuck(A, G, V1,V2,V3))

Error = 1.504309895484143e-13
CPU times: user 61.8 ms, sys: 3.61 ms, total: 65.4 ms
Wall time: 63.7 ms


### Многомерный случай